In [1]:
import json
import torch
import numpy as np
from sklearn.preprocessing import OneHotEncoder

import sys
import warnings
warnings.filterwarnings('ignore')

# Path to local library
sys.path.append('./models')

# Local library
from util import load_data, normalise, train_validate, test

# Random seed
params = json.load(open("hyperparams.json", "r"))
NJOBS = params['njobs']
RANDOM_STATE = params['random_state']
np.random.seed(RANDOM_STATE)
torch.manual_seed(RANDOM_STATE)
torch.cuda.manual_seed(RANDOM_STATE) 
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f"Using {device} device")

Using cuda:0 device


In [2]:
rssi_train, accl_train, location_train = load_data(params['data_path'], control=True)
rssi_test, accl_test, location_test = load_data(params['data_path'], control=False)

train_idx, validation_idx = np.array([]).astype(np.int64), np.array([]).astype(np.int64)
for room in ['dining_room', 'living_room', 'kitchen', 'hall', 'stairs', 'porch/outside_front_door']:
    temp = np.unique(np.argwhere(location_train == room)[:, 0])
    if len(temp):
        train_idx = np.concatenate([train_idx, temp[:int(len(temp)/2)]])
        validation_idx = np.concatenate([validation_idx, temp[int(len(temp)/2):]])
    else:
        print(f'Data for ({room}) is not available!')

rssi_scaler, rssi_train, rssi_test, rssi_val = normalise(rssi_train[train_idx], rssi_test, rssi_train[validation_idx])
accl_scaler, accl_train, accl_test, accl_val = normalise(accl_train[train_idx], accl_test, accl_train[validation_idx])

encoder = OneHotEncoder(sparse=False)
encoder.fit(np.unique(np.concatenate(
    [location_train[train_idx], location_train[validation_idx], location_test]
)).reshape(-1, 1))
location_val = encoder.transform(location_train[validation_idx].reshape(-1, 1)).reshape(len(validation_idx), location_train.shape[1], -1)
location_train = encoder.transform(location_train[train_idx].reshape(-1, 1)).reshape(len(train_idx), location_train.shape[1], -1)
location_test = encoder.transform(location_test.reshape(-1, 1)).reshape(rssi_test.shape[0], rssi_test.shape[1], -1)

print('Train data: {}, Validation data: {}, Test data: {}'.format(
    rssi_train.shape, rssi_val.shape, rssi_test.shape
))

Data for (stairs) is not available!
Data for (porch/outside_front_door) is not available!
Train data: (7289, 25, 20), Validation data: (7290, 25, 20), Test data: (14689, 25, 20)


In [ ]:
model = train_validate(
    torch.from_numpy(rssi_train).type(torch.float).to(device), 
    torch.from_numpy(accl_train).type(torch.float).to(device), 
    torch.from_numpy(location_train).type(torch.float).to(device),
    torch.from_numpy(rssi_val).type(torch.float).to(device),
    torch.from_numpy(accl_val).type(torch.float).to(device),
    torch.from_numpy(location_val).type(torch.float).to(device), 
    np.argmax(encoder.transform([['hall']])), params
)

Batch: 0, Train Loss: 47.55542, Acc: 0.00535
Batch: 5120, Train Loss: 45.57110, Acc: 0.30333
Val Acc: 0.61427
Batch: 0, Train Loss: 10.57161, Acc: 0.13280
Batch: 5120, Train Loss: 21.01987, Acc: 0.60077
Val Acc: 0.75117
Batch: 0, Train Loss: 11.16845, Acc: 0.13267
Batch: 5120, Train Loss: 18.28908, Acc: 0.71944
Val Acc: 0.76543
Batch: 0, Train Loss: 12.90567, Acc: 0.13212
Batch: 5120, Train Loss: 15.22618, Acc: 0.76320
Val Acc: 0.76598
Batch: 0, Train Loss: 13.95231, Acc: 0.13088
Batch: 5120, Train Loss: 14.03514, Acc: 0.77157
Val Acc: 0.81166
Batch: 0, Train Loss: 14.65258, Acc: 0.12965
Batch: 5120, Train Loss: 12.42518, Acc: 0.78474
Val Acc: 0.95336
Batch: 0, Train Loss: 16.62818, Acc: 0.11620
Batch: 5120, Train Loss: 11.57179, Acc: 0.77500
Val Acc: 0.97311
Batch: 0, Train Loss: 16.54593, Acc: 0.11593
Batch: 5120, Train Loss: 10.60956, Acc: 0.78063
Val Acc: 0.98066
Batch: 0, Train Loss: 15.59942, Acc: 0.11277
Batch: 5120, Train Loss: 9.13381, Acc: 0.78735
Val Acc: 0.98038
Batch: 0, T

In [ ]:
test(
    model, 
    torch.from_numpy(rssi_test).type(torch.float).to(device), 
    torch.from_numpy(accl_test).type(torch.float).to(device), 
    torch.from_numpy(location_test).type(torch.float).to(device), 
    params
)